In [1]:
#importing what is needed
import time
import pandas as pd
import requests
import json5


In [2]:
url_base = 'https://www.indeed.com/jobs?q=machine%20learning&l=remote&explvl=entry_level&fromage=1&limit=50'

In [3]:
jobs = []

In [4]:
err_count = 0
r = requests.get(url_base)
while r.status_code!=200 and err_count < 5:
    time.sleep(5)
    err_count = err_count + 1
    r = requests.get(url_base)
if err_count==5:
    quit()
text = r.text
for i in range(text.count('jobmap[')):
    start = text.find('jobmap[{}]'.format(i))
    if start==-1:
        continue
    end = text.find('jobmap[{}]'.format(i+1))
    s = text.find('{',start)
    if end == -1:
        e = text.find('}',s)
    else:
        e = text.rfind('}',s,end)
    job = json5.loads(text[s:e+1])
    jobs.append(job)
   

In [6]:
df = pd.DataFrame(jobs)
df.drop_duplicates(inplace=True)
df['job_link']='https://www.indeed.com/viewjob?jk='+df.jk
df.rename(columns={'cmp':'company'},inplace=True)
df = df[(df['country']=="US")&(df["loc"]=="Remote")].copy()

In [7]:
df.head()

,jk,efccid,srcid,cmpid,num,srcname,company,cmpesc,cmplnk,loc,country,zip,city,title,locid,rd,job_link
0,94ee20b008c76f24,caff23281376b83d,c6c40f68ed4ae948,1861a7eb651b86ce,0,Amazon.com,Amazon.com Services LLC,Amazon.com Services LLC,/q-Amazon.com-Services-l-remote-jobs.html,Remote,US,,Remote,Virtual Bilingual Machine Learning Data Associ...,aaa2b906602aa8f5,0HcsNzDq-OLJ6bT4SC-8zOADK7b00hFO4bREt9KrzqfaM7...,https://www.indeed.com/viewjob?jk=94ee20b008c7...
1,92e1899657f8959d,dff5ec7b84dd09cf,69a0a8cc667a42c8,5f044e418a82db6c,1,Indeed,TEEMA Solutions Group Inc,TEEMA Solutions Group Inc,/q-TEEMA-Solutions-Group-l-remote-jobs.html,Remote,US,,Remote,Machine Learning Engineer - 100% Remote,aaa2b906602aa8f5,twcb2ZEJnz8JwcbwiR9wTfmQAEoTlrC6F-NzPOae7iF7x2...,https://www.indeed.com/viewjob?jk=92e1899657f8...
2,9ffa653167403a99,a120bc13f2aa6f20,7fbdd9ef62707c8e,ce8c5c4d60bb4cad,2,Cambium Learning Group,Cambium Learning Group,Cambium Learning Group,/q-Cambium-Learning-Group-l-remote-jobs.html,Remote,US,,Remote,Psychometric Support/Research Assistant,aaa2b906602aa8f5,fmQo99wCEscDVgmVOMcwfCA6mI6xhKRFdgVZF9Rf_QKW-3...,https://www.indeed.com/viewjob?jk=9ffa65316740...
3,730812239c80d3c3,acaeeb8e1013843e,aba680cd210c46f3,fdff5ca28467e6dd,3,PTC THERAPEUTICS,PTC THERAPEUTICS,PTC THERAPEUTICS,/q-PTC-THERAPEUTICS-l-remote-jobs.html,Remote,US,,Remote,"Contract, Machine Learning Engineering Fellow",aaa2b906602aa8f5,2xoO0TjyPVRYyCMAB-zZvdkmQzRlY6lfV_KNuwFoMDiDeh...,https://www.indeed.com/viewjob?jk=730812239c80...
4,5bb5d53eedf30728,916965801c450315,8079a70794ccfe77,8ce86f98f92474e7,4,Indeed,"Trobus Technologies,LLC","Trobus Technologies,LLC",/q-Trobus-Technologies-l-remote-jobs.html,Remote,US,,Remote,Data Scientist,aaa2b906602aa8f5,ID3r8zo9PR2y-cNXBZlqKS6_DopDOIwJdxrcabGJcHyz2W...,https://www.indeed.com/viewjob?jk=5bb5d53eedf3...


In [13]:
df.reset_index(inplace=True)

In [14]:
html = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width">
  <title>Remote jobs</title>
</head>
<body>
<h1>Remote job listings</h1>
<ul>
"""
html_end = """
</ul>
</body>
</html>
"""
for i in range(df.shape[0]):
    line = '<li><a href="{}">{}: {}</a></li>'.format(df.loc[i,'job_link'], df.loc[i,'company'], df.loc[i,'title'])
    html += line
html+=html_end

In [9]:
html

'\n<!DOCTYPE html>\n<html>\n<head>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width">\n  <title>Remote jobs</title>\n</head>\n<body>\n<h1>Remote job listings</h1>\n<ul>\n<li><a href="https://www.indeed.com/viewjob?jk=94ee20b008c76f24">Amazon.com Services LLC: Virtual Bilingual Machine Learning Data Associate (US)- German</a></li><li><a href="https://www.indeed.com/viewjob?jk=92e1899657f8959d">TEEMA Solutions Group Inc: Machine Learning Engineer - 100% Remote</a></li><li><a href="https://www.indeed.com/viewjob?jk=9ffa653167403a99">Cambium Learning Group: Psychometric Support/Research Assistant</a></li><li><a href="https://www.indeed.com/viewjob?jk=730812239c80d3c3">PTC THERAPEUTICS: Contract, Machine Learning Engineering Fellow</a></li><li><a href="https://www.indeed.com/viewjob?jk=5bb5d53eedf30728">Trobus Technologies,LLC: Data Scientist</a></li>'

In [11]:
df.iloc[5]

jk                                           ac5fdf9ea6888067
efccid                                       8c4e22bfa505728b
srcid                                        3dad6d3ad2aae1cd
cmpid                                        c594c442f5397e7b
num                                                         6
srcname                                                GitHub
company                                                Github
cmpesc                                                 Github
cmplnk                           /q-Github-l-remote-jobs.html
loc                                                    Remote
country                                                    US
zip                                                          
city                                                   Remote
title            Staff Software Engineer - Strategic Programs
locid                                        aaa2b906602aa8f5
rd          PWfcGNFA7tgIDxGVBcVa2IqZjfm4--rgPGwbj1lNDvSDa2...
job_link